In [ ]:
import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import mne
import networkx as nx
import h5py
from scipy.io import savemat, loadmat
from autoreject.autoreject import _apply_interp


from data_analysis.functions_preprocessing import \
    (combine_raws, split_raws, combine_epochs, split_epochs,
     preprocess_single_sub, load_ica, load_autoreject)
from data_analysis.functions_behavioral import \
    (create_event_df, remove_ghost_triggers, calculate_alpha,
     join_event_dfs, remove_outliers, events_from_event_df)

# number of cores to use for parallel processing (ramsauer pc should have 80 cores)
n_jobs = 7

subject_dir = "/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/"
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"
result_dir = "/net/store/nbp/projects/hyperscanning/study_project/results"


# Main Data Analysis ###################################


# Perform the data analysis
for subj_pair in ['204']:

    subs_path = subject_dir + "sub-{0}/eeg/sub-{0}_task-hyper_eeg.fif".format(subj_pair)
    behav_path = op.join(behav_dir, "{0}.csv".format(subj_pair))

    combined_raw = mne.io.read_raw_fif(subs_path, preload=True)

    # split the subjects and delete the raw file
    raws = split_raws(combined_raw)
    del combined_raw
    
    for i, _ in enumerate(raws):
        # set the EEG Montage. We use 64 chans from the standard 10-05 system.
        montage = mne.channels.make_standard_montage("standard_1005")
        raws[i].set_montage(montage)

    # combine the subjects again
    raw_combined = combine_raws(raws[0], raws[1])
    del raws  # to save memory
    
    # filter
    raw_combined.filter(l_freq=0.1, h_freq=120)
    
    # define the epoching window
    tmin = 0
    tmax = 1.5

    # do the behavioral analysis and get the epochs
    behavioral_df = calculate_alpha(pd.read_csv(behav_path))
    event_df = create_event_df(raw_combined)
    event_df = remove_ghost_triggers(event_df)
    event_df = join_event_dfs(event_df, behavioral_df)
    
    # get the first tap by looking at the first sample in each trial
    min_idx = event_df.groupby(["trial"])["sample"].idxmin()
    early_df = event_df[event_df.index.isin(min_idx)]
    early_events = events_from_event_df(early_df)
    early_events[:,-1] = 1
    
    # get the late taps by looking at the last sample - 1.5 seconds
    max_idx = event_df.groupby(["trial"])["sample"].idxmax()
    late_df = event_df[event_df.index.isin(max_idx)]
    late_events = events_from_event_df(late_df)
    late_events[:,0] -= int(raw_combined.info["sfreq"] * 1.5)
    late_events[:,-1] = 2
    
    # get the baseline events (an equally scaled window right before the early epochs)
    base_events = early_events.copy()
    base_events[:,0] -= int(raw_combined.info["sfreq"] * (tmax - tmin))
    base_events[:,-1] = 0

    # define the parameters for epoching
    combined_events = np.vstack([base_events, early_events, late_events])

    # epoch the data. Here we filter out bad segments from both participants
    epochs = mne.Epochs(raw_combined, combined_events, tmin=tmin, tmax=tmax,
                        picks=["eeg"], baseline=(0, 0), preload=True) # only use the first two epochs
    epochs.event_id = dict(baseline=0, early=1, late=2)
    
    
    # we have to combine both autoreject thresholds first and remove the manually
    #rejects = [load_autoreject("sub-{0}_p-{1}".format(subj_pair, i)).get_reject_log(split_epochs(epochs)[i]).bad_epochs for i in range(2)]
    #combined_rejects = np.logical_or(rejects[0], rejects[1])
    
    event_types = ["baseline", "early", "late"]
    

    rejects = []
    for i in range(2):
        cond_rejects = []
        for condition in event_types:
            subj_id = "sub-{0}_p-{1}-{2}".format(subj_pair, i, condition)
            cur_reject = load_autoreject(subj_id).get_reject_log(split_epochs(epochs)[i][condition]).bad_epochs
            cond_rejects.append(cur_reject)
        
        rejects.append(np.logical_or(np.logical_or(cond_rejects[0], cond_rejects[1]), cond_rejects[2]))
    combined_rejects = np.hstack([np.logical_or(rejects[0], rejects[1]) for i in range(3)])

    
    #rejects = np.hstack([[load_autoreject("sub-{0}_p-{1}-{2}".format(subj_pair, i, condition)).get_reject_log(split_epochs(epochs[condition])[i]).bad_epochs for i in range(2)] for condition in event_types])
    #combined_rejects = np.logical_or(rejects[0], rejects[1])
    
    # apply the heuristic to reject all parts of a trial if 2 or more epochs out of
    # baseline, early, and late, are bad.
    #bad_trials = np.vstack([combined_rejects[:300],
    #                        combined_rejects[300:600],
    #                        combined_rejects[600:]])
    #bad_trial_sets = np.sum(bad_trials, axis=0) >= 1
    #combined_rejects = np.hstack([bad_trial_sets] * 3)
    
    epochs = epochs.drop(combined_rejects, reason="Autoreject")
    
    # save the dropped epochs
    drop_list = [ind for ind, val in enumerate(epochs.drop_log[:300]) if val == ["Autoreject"]]
    drop_fname = op.join(result_dir, "dropped_epochs", subj_pair + ".mat")
    savemat(drop_fname, {"drop_list":drop_list})

    
     # split the epochs to apply ICA and TFR transform
    epochs_split = list(split_epochs(epochs))
    
    # frequencies should be 25 freqs, log spaced between 4 and 50
    freqs = np.logspace(np.log10(4), np.log10(45), 20)#[:2] # TODO: only here for debugging! remove again!
    cycles = freqs / 2.
    
    for i, cur_eps in enumerate(epochs_split):
        subj_id = "sub-{0}_p-{1}".format(subj_pair, i)
        
        condition_split = []
        for condition in event_types:
        
            cur_cond_eps = cur_eps[condition]
            
            # apply autoreject (exclude bads and interpolate)
            # TODO: The Autoreject guys apply ICA first and then autoreject local. i would do the same
            # Applying ICA first will look ugly, but for the first pair, it saves ~ 50 epochs
            ar = load_autoreject(subj_id + "-" + condition)
            reject_log = ar.get_reject_log(cur_cond_eps)
            #cur_cond_eps = ar.transform(cur_cond_eps, return_log=False)

            # apply ICA
            ica = load_ica(subj_id)
            cur_cond_eps = ica.apply(cur_cond_eps)
            
            # interpolate channels from autoreject
            _apply_interp(reject_log, cur_cond_eps, ar.threshes_,
                          ar.picks_, ar.dots, ar.verbose)
            
            condition_split.append(cur_cond_eps)
            
        # concatenate the different conditions again
        cur_eps = mne.concatenate_epochs(condition_split)
        
        # rereference to avg ref
        cur_eps.set_eeg_reference(ref_channels='average')

        # apply surface laplacian
        cur_eps = mne.preprocessing.compute_current_source_density(cur_eps,
                                                                   stiffness=4,
                                                                   lambda2=1e-5)
        
        epochs_split[i] = cur_eps
        
        
    #combine the epochs again
    epochs = combine_epochs(epochs_split[0], epochs_split[1])
    
    
                         
    for condition in event_types:
    
        # Get the phase angles via a wavelet transform
        phases = mne.time_frequency.tfr_morlet(epochs[condition], freqs, cycles, output="phase",
                                               return_itc=False, average=False, n_jobs=n_jobs)
        
        # save the condition
        fname = op.join(result_dir, "phase_angles", subj_pair + "_" + condition)
        phases.save(fname)
        del phases
        
    # subtract the baseline
    #phase_angles["early"].data -= phase_angles["baseline"].data
    #phase_angles["late"].data -= phase_angles["baseline"].data
    
    # delete some stuff to free some memory
    #del phase_angles["baseline"]
    del epochs
    del epochs_split
    del cur_eps
    del condition_split
    del cur_cond_eps
    del raw_combined
    
    

Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-204/eeg/sub-204_task-hyper_eeg.fif...


<ipython-input-5-6d79c5da5c0a>:40: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-204/eeg/sub-204_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3537048 =      0.000 ...  3454.148 secs
Ready.
Reading 0 ... 3537048  =      0.000 ...  3454.148 secs...
EEG channel type selected for re-referencing
Applying a custom EEG reference.
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 33793 samples (33.001 sec)

6638 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 3

<ipython-input-5-6d79c5da5c0a>:90: RuntimeWarning: The events passed to the Epochs constructor are not chronologically ordered.
  epochs = mne.Epochs(raw_combined, combined_events, tmin=tmin, tmax=tmax,


0 bad epochs dropped
Dropped 522 epochs: 0, 1, 2, 3, 4, 5, 7, 9, 10, 11, 12, 15, 16, 17, 18, 19, 20, 22, 23, 24, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56, 57, 59, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 75, 76, 77, 78, 84, 85, 86, 90, 92, 93, 94, 96, 97, 98, 99, 100, 101, 102, 103, 107, 108, 109, 110, 111, 113, 114, 115, 117, 118, 121, 122, 123, 125, 126, 127, 128, 130, 131, 132, 133, 134, 135, 136, 138, 139, 142, 145, 150, 151, 152, 153, 155, 156, 157, 160, 167, 168, 170, 171, 173, 175, 177, 178, 179, 180, 181, 183, 186, 190, 194, 198, 203, 204, 205, 208, 213, 216, 217, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 233, 235, 237, 239, 241, 244, 245, 246, 248, 250, 251, 253, 254, 255, 257, 260, 267, 269, 273, 276, 277, 279, 280, 281, 284, 290, 296, 299, 300, 301, 302, 303, 304, 305, 307, 309, 310, 311, 312, 315, 316, 317, 318, 319, 320, 322, 323, 324, 327, 328, 329, 330, 331, 332, 334, 335, 336, 337, 338, 340, 342, 343, 

...

Not setting metadata


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:    7.5s
[Parallel(n_jobs=7)]: Done  65 tasks      | elapsed:   35.2s
[Parallel(n_jobs=7)]: Done 128 out of 128 | elapsed:  1.1min finished


Not setting metadata


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:    7.6s
[Parallel(n_jobs=7)]: Done  65 tasks      | elapsed:   35.2s
[Parallel(n_jobs=7)]: Done 128 out of 128 | elapsed:  1.1min finished


Not setting metadata


In [4]:
len(drop_list)

174

In [1]:
import os.path as op
import mne
result_dir = "/net/store/nbp/projects/hyperscanning/study_project/results"
subj_pair = "202"
condition = "early"
fname = op.join(result_dir, "phase_angles", subj_pair + "_" + condition)

epochs = mne.time_frequency.read_tfrs(fname)[0]


Reading /net/store/nbp/projects/hyperscanning/study_project/results/phase_angles/202_early ...


<ipython-input-1-134625a7d2dd>:8: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/study_project/results/phase_angles/202_early) does not conform to MNE naming conventions. All tfr files should end with -tfr.h5 or _tfr.h5
  epochs = mne.time_frequency.read_tfrs(fname)


Not setting metadata


In [1]:
epochs

NameError: name 'epochs' is not defined

In [ ]:


    
     # split the epochs to apply ICA and TFR transform
    epochs_split = list(split_epochs(epochs))
    
    # frequencies should be 25 freqs, log spaced between 4 and 50
    freqs = np.logspace(np.log10(4), np.log10(45), 20)#[:2] # TODO: only here for debugging! remove again!
    cycles = freqs / 2.
    
    for i, cur_eps in enumerate(epochs_split):
        subj_id = "sub-{0}_p-{1}".format(subj_pair, i)
        
        condition_split = []
        for condition in event_types:
        
            cur_cond_eps = cur_eps[condition]
            
            # apply autoreject (exclude bads and interpolate)
            # TODO: The Autoreject guys apply ICA first and then autoreject local. i would do the same
            # Applying ICA first will look ugly, but for the first pair, it saves ~ 50 epochs
            ar = load_autoreject(subj_id + "-" + condition)
            reject_log = ar.get_reject_log(cur_cond_eps)
            #cur_cond_eps = ar.transform(cur_cond_eps, return_log=False)

            # apply ICA
            ica = load_ica(subj_id)
            cur_cond_eps = ica.apply(cur_cond_eps)
            
            # interpolate channels from autoreject
            _apply_interp(reject_log, cur_cond_eps, ar.threshes_,
                          ar.picks_, ar.dots, ar.verbose)
            
            condition_split.append(cur_cond_eps)
            
        # concatenate the different conditions again
        cur_eps = mne.concatenate_epochs(condition_split)
        
        # rereference to avg ref
        cur_eps.set_eeg_reference(ref_channels='average')

        # apply surface laplacian
        cur_eps = mne.preprocessing.compute_current_source_density(cur_eps,
                                                                   stiffness=4,
                                                                   lambda2=1e-5)
        
        epochs_split[i] = cur_eps
        
        
    #combine the epochs again
    epochs = combine_epochs(epochs_split[0], epochs_split[1])
    
    
    # initialize containers to analyze later
    phase_angles = {}
    ispc_matrices = {}
    small_worlds = {}
                         
    for condition in event_types:
    
        # Get the phase angles via a wavelet transform
        phases = mne.time_frequency.tfr_morlet(epochs[condition], freqs, cycles, output="phase",
                                               return_itc=False, average=False, n_jobs=n_jobs)
        phase_angles[condition] = phases#[:8] # TODO: only here for debugging! remove again!
        
    # subtract the baseline
    phase_angles["early"].data -= phase_angles["baseline"].data
    phase_angles["late"].data -= phase_angles["baseline"].data
    
    # delete some stuff to free some memory
    del phase_angles["baseline"]
    del epochs
    del epochs_split
    del cur_eps
    del condition_split
    del cur_cond_eps
    del raw_combined
    
    # save the data
    for condition in ["early", "late"]:
        
        fname = op.join(result_dir, "phase_angles", subj_pair + "_" + condition + ".hdf5")
        h5f = h5py.File(fname, "w")
        h5f.create_dataset(condition, data=phase_angles[condition].data)
        h5f.close()
            
        savemat(op.join(result_dir, "phase_angles", subj_pair + "_times.mat"),
                {"times":phase_angles[condition].times})
        del phase_angles[condition]



In [1]:
import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import mne
import h5py
from scipy.io import savemat, loadmat

from data_analysis.functions_connectivity import \
    epochs_ispc, multi_ispc
from data_analysis.functions_graph_theory import \
    multi_small_world

# define which ISPCS to calculate
#['202','203','204','205','206','207','208','209','211','212']
subj_pairs = ['203']

# conditions ["early", "late"]
conditions = ["early"]

# number of cores to use for parallel processing (ramsauer pc should have 80 cores)
n_jobs = 31

subject_dir = "/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/"
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"
result_dir = "/net/store/nbp/projects/hyperscanning/study_project/results"


# calculate the ISPC ###################################
for subj_pair in subj_pairs:
    
    # get the baseline
    baseline = mne.time_frequency.read_tfrs(op.join(result_dir, "phase_angles", subj_pair + "_baseline"))[0]
    
    for condition in conditions:
        print("Calculating ISPCs for: {0}, condition {1}".format(subj_pair, condition), flush=True)
        
        # load the phase_angles from data
        phase_angles = mne.time_frequency.read_tfrs(op.join(result_dir, "phase_angles", subj_pair + "_" + condition + ""))[0]
        
        # subtract the baseline
        phase_angles.data -= baseline.data
        
        # ISPC
        ispc_matrices = multi_ispc(phase_angles, n_jobs=n_jobs)
        
        # save the first batch of data
        savemat(op.join(result_dir, "ispc_matrices", subj_pair + "_" + condition + ".mat"),
                {condition:ispc_matrices})
        del phase_angles
        del ispc_matrices
        

In /net/store/nbp/projects/hyperscanning/study_project/programming_tools/miniconda3/envs/hyperscanning/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /net/store/nbp/projects/hyperscanning/study_project/programming_tools/miniconda3/envs/hyperscanning/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /net/store/nbp/projects/hyperscanning/study_project/programming_tools/miniconda3/envs/hyperscanning/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /net/store/nbp/projects/hyperscanning/study_project

Reading /net/store/nbp/projects/hyperscanning/study_project/results/phase_angles/203_baseline ...


<ipython-input-1-89c502e6a74c>:36: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/study_project/results/phase_angles/203_baseline) does not conform to MNE naming conventions. All tfr files should end with -tfr.h5 or _tfr.h5
  baseline = mne.time_frequency.read_tfrs(op.join(result_dir, "phase_angles", subj_pair + "_baseline"))[0]


Not setting metadata
Calculating ISPCs for: 203, condition early
Reading /net/store/nbp/projects/hyperscanning/study_project/results/phase_angles/203_early ...


<ipython-input-1-89c502e6a74c>:42: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/study_project/results/phase_angles/203_early) does not conform to MNE naming conventions. All tfr files should end with -tfr.h5 or _tfr.h5
  phase_angles = mne.time_frequency.read_tfrs(op.join(result_dir, "phase_angles", subj_pair + "_" + condition + ""))[0]


Not setting metadata
